# **Import Library**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import re
import os
import kagglehub
from kagglehub import KaggleDatasetAdapter

# **Function Library**

In [2]:
def extract_prices(val):
    """Converts string prices to numerical prices (eg. '$1,070.23' --> 1070.23) """

    # Return value if null
    if pd.isna(val):
        return np.nan

    # Remove whitespace
    s = str(val).strip()

    # Extract value after the $ sign
    match = re.search(r"\$?\s*([0-9]*,?[0-9]*\.?[0-9]+)", s)
    if not match:
        return np.nan

    # Remove commas
    num_str = match.group(1).replace(",", "")

    return float(num_str)

def extract_main_category(val):
    """Extracts the main category (eg. 'Electronics | Computers & Accessories' --> "Electronics") """
    
    # Return value if null
    if pd.isna(val):
        return None

    # Remove whitespace
    s = str(val).strip()
    
    # Extract main category
    main_cat = s.split(" | ")[0].strip().lower()


    return main_cat

def extract_weights(val):
    """Converts string weights to numerical pounds (eg. '16 ounces --> 0.5) """

    # Return value if null
    if pd.isna(val):
        return np.nan
    
    # Remove commas and trim whitespace
    val = str(val).strip().replace(",", "")
    
    # Extract the numerical weight and weight type
    match = re.match(r"([0-9]*\.?[0-9]+)\s*(pounds|pound|lbs|ounces|ounce|oz)", val.lower())
    if not match:
        return np.nan
    num = float(match.group(1))
    unit = match.group(2)

    # Convert to pounds if in ounces
    if unit in ["ounces", "ounce", "oz"]:
        return num / 16.0
    else:
        return num



def extract_dimensions(text):
    """Extract dimensions from string (eg. '14.7 x 11.1 x 10.2 inches')"""
    
    # Return value if null
    if not isinstance(text, str):
        return pd.Series([np.nan, np.nan, np.nan, np.nan])

    # Regex to capture L x W x H with optional unit
    dim_pattern = re.compile(
        r'(\d+\.?\d*)\s*[xX×]\s*(\d+\.?\d*)\s*(?:[xX×]\s*(\d+\.?\d*))?\s*(inches|inch|in)?',
        re.IGNORECASE
    )

    # Try match without spaces (handles formats like "3.5x6.2x13inches")
    match = dim_pattern.search(text.replace(" ", ""))

    # If no match, try again preserving spaces (handles "14.7 x 11.1 x 10.2 inches")
    if not match:
        match = dim_pattern.search(text)

    # If still no match → no dimension found
    if not match:
        return pd.Series([np.nan, np.nan, np.nan, np.nan])

    # Extract length, width, height
    length = float(match.group(1))
    width = float(match.group(2))
    height = match.group(3)
    height = float(height) if height else np.nan

    # Clean and standardize unit (default → inches)
    unit = match.group(4)
    if unit:
        unit = unit.lower()
        unit = "inches" if unit in ["inch", "in"] else unit
    else:
        unit = "inches"

    return pd.Series([length, width, height, unit])

def extract_brand(product_name):
    """
    Extract brand from the beginning of a product name.
    
    Rules:
    - Take the leading capitalized word(s)
    - Stop before digits, lower-case transitions, or model numbers
    - If a hyphenated prefix exists, take left side
    """

    # Return value if null
    if not isinstance(product_name, str):
        return np.nan

    # Remove whitespace
    name = product_name.strip()

    # If hyphen early it is likely brand prefix
    if "-" in name.split(" ")[0]:
        return name.split(" ")[0].split("-")[0]

    # Tokenize the name
    tokens = name.split()

    brand_tokens = []
    for token in tokens:
        # Stop if token begins with a digit
        if re.match(r'^\d', token):
            break
        
        # Stop if lowercase indicates description not brand
        if token[0].islower():
            break
        
        # Stop if token looks like a model number
        if re.match(r'^[A-Za-z]*\d+', token):  
            break
        
        brand_tokens.append(token)

    if len(brand_tokens) == 0:
        return np.nan

    return " ".join(brand_tokens)

def extract_price_per(price, weight_lb):
    """Computes price per pound. """
    
    # Return if value is null
    if pd.isna(price) or pd.isna(weight_lb):
        return np.nan
    
    return price / weight_lb

def clean_text(x):
    """Cleans text (eg. removing common verbiage, collapsing whitespace, lowercase optional). """
    if not isinstance(x, str):
        return ""
    
    # Remove Amazon boilerplate
    x = re.sub(r"make sure this fits by entering your model number\.?", "", x, flags=re.IGNORECASE)
    
    # Remove separators like " | "
    x = x.replace("|", " ")
    
    # Remove multiple spaces
    x = re.sub(r"\s+", " ", x).strip()
    
    return x

def remove_ui_noise(text):
    """
    Cleans out Amazon UI boilerplate and scraped metadata blocks.
    Removes:
    - shipping weight lines
    - product dimensions lines (scraped, not structured)
    - ASIN, item model numbers, manufacturer age ranges
    - domestic/international shipping messages
    - style attributes
    - glued-together UI artifacts
    """

    noise_patterns = [
        r"View shipping rates and policies",
        r"Go to your orders and start the return",
        r"Select the ship method",
        r"Ship it!",
        r"ASIN:\S+",
        r"Itemmodelnumber:\S+",
        r"Manufacturerrecommendedage:\S+",
        r"ProductDimensions:\S+",
        r"ItemWeight:\S+",
        r"ShippingWeight:\S+",
        r"DomesticShipping:\S+",
        r"InternationalShipping:\S+",
        r"Style:\S+",
        r"LearnMore",
        r"\(\)",                   # empty parentheses
        r"\d+\.?\d*\s*ounces",     # any weight in ounces
        r"\d+\.?\d*\s*inch(es)?",  # bare dimensions
    ]

    for pat in noise_patterns:
        text = re.sub(pat, " ", text, flags=re.IGNORECASE)

    # Collapse any leftover whitespace
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def extract_embedding(row):
    """
    Builds a merged text field for product embeddings.

    Includes:
    - product_name
    - brand_name
    - category + category path
    - about_product
    - product_specification
    - technical_details
    - dimensions (if available)
    - price per pound (human readable)

    Excludes:
    - Amazon UI text (shipping boilerplate)
    - repeated glue text
    """

    parts = []

    # Product name
    if pd.notna(row.get("product_name")):
        parts.append(clean_text(row["product_name"]))

    # Brand
    if pd.notna(row.get("brand_name")):
        parts.append(f"Brand: {clean_text(row['brand_name'])}")

    # Categories
    if pd.notna(row.get("main_category")):
        parts.append(f"Category: {clean_text(row['main_category'])}")

    if pd.notna(row.get("category")):
        parts.append(f"Category path: {clean_text(row['category'])}")

    # About product
    if pd.notna(row.get("about_product")):
        cleaned = clean_text(row["about_product"])
        cleaned = remove_ui_noise(cleaned)
        parts.append(cleaned)

    # Product specification
    if pd.notna(row.get("product_specification")):
        cleaned = clean_text(row["product_specification"])
        cleaned = remove_ui_noise(cleaned)
        parts.append(cleaned)

    # Technical details
    if pd.notna(row.get("technical_details")):
        cleaned = clean_text(row["technical_details"])
        cleaned = remove_ui_noise(cleaned)
        parts.append(cleaned)

    # Dimensions
    dims = []
    if pd.notna(row.get("dim_length")):
        dims.append(f"length {row['dim_length']}")
    if pd.notna(row.get("dim_width")):
        dims.append(f"width {row['dim_width']}")
    if pd.notna(row.get("dim_height")):
        dims.append(f"height {row['dim_height']}")

    if dims:
        unit = row.get("dim_unit", "")
        if isinstance(unit, str) and len(unit) > 0:
            parts.append("Dimensions: " + ", ".join(dims) + f" {unit}")
        else:
            parts.append("Dimensions: " + ", ".join(dims))

    # Price per pound
    if pd.notna(row.get("normalized_weight")):
        price_val = row["normalized_weight"]
        readable = f"Price per pound: ${price_val:.2f}"
        parts.append(readable)

    # Join everything
    full_text = " ".join(parts)
    full_text = re.sub(r"\s+", " ", full_text).strip()

    return full_text

def extract_metadata(row):
    """
    Build a metadata dictionary for a product row.

    Includes:
    - Core identifiers
    - Category information
    - Numeric metadata
    - Feature text (combined)
    - Optional ingredients
    - Flags / URLs / quality indicators
    """

    # Construct a minimal "features" field using your available descriptive fields
    features_text = " ".join([
        str(row.get("about_product", "")),
        str(row.get("product_specification", "")),
        str(row.get("technical_details", "")),
    ]).strip()

    metadata_dict = {
        # Identifiers
        "product_id": row.get("product_id"),
        "product_name": row.get("product_name"),

        # Categories
        "main_category": row.get("main_category"),
        "category_path": row.get("category"),

        # Numeric metadata
        "price": row.get("selling_price"),
        "shipping_weight_lbs": row.get("shipping_weight"),
        "dim_length": row.get("dim_length"),
        "dim_width": row.get("dim_width"),
        "dim_height": row.get("dim_height"),

        # Flags
        "is_amazon_seller": row.get("is_amazon_seller", False),  # Already bool earlier
        "has_variants": pd.notna(row.get("variants")),

        # Text metadata
        "model_number": row.get("model_number"),
        "brand": row.get("brand_name"),   # Use your extracted brand_name field
        "image_url": row.get("image"),
        "product_url": row.get("product_url"),

        # Computed metadata
        "features": features_text,
        "ingredients": None,  # Dataset doesn't include structured ingredients

        # Length / quality indicators
        "about_length": len(row["about_product"]) if pd.notna(row.get("about_product")) else 0,
        "spec_length": len(row["product_specification"]) if pd.notna(row.get("product_specification")) else 0,
        "tech_details_length": len(row["technical_details"]) if pd.notna(row.get("technical_details")) else 0,
    }

    return metadata_dict

# **Data Loading**

## Load Products Dataset

In [3]:
# Read dataset from HF
products = pd.read_csv("hf://datasets/calmgoose/amazon-product-data-2020/amazon_product_data_cleaned.csv")
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Uniq Id                10002 non-null  object
 1   Product Name           10002 non-null  object
 2   Category               9172 non-null   object
 3   Upc Ean Code           34 non-null     object
 4   Selling Price          9895 non-null   object
 5   Model Number           8232 non-null   object
 6   About Product          9729 non-null   object
 7   Product Specification  8370 non-null   object
 8   Technical Details      9212 non-null   object
 9   Shipping Weight        8864 non-null   object
 10  Product Dimensions     479 non-null    object
 11  Image                  10002 non-null  object
 12  Variants               2478 non-null   object
 13  Product Url            10002 non-null  object
 14  Is Amazon Seller       10002 non-null  object
dtypes: object(15)
memor

## Load Reviews Dataset

In [4]:
path = kagglehub.dataset_download("promptcloud/amazon-product-reviews-dataset")
reviews = pd.read_csv(os.path.join(path, [f for f in os.listdir(path) if f.endswith(".csv")][0]))
reviews.info()

100%|██████████| 12.5k/12.5k [00:00<00:00, 11.6MB/s]

Extracting files...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Uniq Id                50 non-null     object 
 1   Crawl Timestamp        50 non-null     object 
 2   Billing Uniq Id        50 non-null     object 
 3   Rating                 0 non-null      float64
 4   Review Title           50 non-null     object 
 5   Review Rating          50 non-null     float64
 6   Review Date            50 non-null     object 
 7   User Id                50 non-null     object 
 8   Brand                  50 non-null     object 
 9   Category               49 non-null     object 
 10  Sub Category           49 non-null     object 
 11  Product Description    49 non-null     object 
 12  Asin                   50 non-null     object 
 13  Url                    50 non-null     object 
 14  Review Content         50 non-null     o

## Merge Datasets

In [5]:
product_ids = set(products["Uniq Id"])
review_ids = set(reviews["Uniq Id"])

print("Product IDs:", len(product_ids))
print("Review IDs:", len(review_ids))
print("Common IDs:", len(product_ids & review_ids))

# Show a few examples
print("Some intersections:", list(product_ids & review_ids))

Product IDs: 10002
Review IDs: 50
Common IDs: 0
Some intersections: []


In [6]:
df = products.merge(reviews, on= 'Uniq Id', how= 'left')
df = df.drop(columns=["Category_y"]).rename(columns={"Category_x": "Category"})
df = df.drop_duplicates(subset=["Uniq Id", "Model Number"], keep= "first")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10002 entries, 0 to 10001
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Uniq Id                10002 non-null  object 
 1   Product Name           10002 non-null  object 
 2   Category               9172 non-null   object 
 3   Upc Ean Code           34 non-null     object 
 4   Selling Price          9895 non-null   object 
 5   Model Number           8232 non-null   object 
 6   About Product          9729 non-null   object 
 7   Product Specification  8370 non-null   object 
 8   Technical Details      9212 non-null   object 
 9   Shipping Weight        8864 non-null   object 
 10  Product Dimensions     479 non-null    object 
 11  Image                  10002 non-null  object 
 12  Variants               2478 non-null   object 
 13  Product Url            10002 non-null  object 
 14  Is Amazon Seller       10002 non-null  object 
 15  Cr

# **Data Preprocessing**

## Typing

In [7]:
df = df.rename(columns={
    
    # Product fields
    "Uniq Id": "product_id",
    "Product Name": "product_name",
    "Category": "category",
    "Upc Ean Code": "upc_ean_code",
    "Selling Price": "selling_price",
    "Model Number": "model_number",
    "About Product": "about_product",
    "Product Specification": "product_specification",
    "Technical Details": "technical_details",
    "Shipping Weight": "shipping_weight",
    "Product Dimensions": "product_dimensions",
    "Image": "image",
    "Variants": "variants",
    "Product Url": "product_url",
    "Is Amazon Seller": "is_amazon_seller",

    # Review fields
    "Crawl Timestamp": "crawl_timestamp",
    "Billing Uniq Id": "billing_id",
    "Rating": "rating",
    "Review Title": "review_title",
    "Review Rating": "review_rating",
    "Review Date": "review_date",
    "User Id": "user_id",
    "Brand": "brand",
    "Sub Category": "sub_category",
    "Product Description": "product_description",
    "Asin": "asin",
    "Url": "review_url",
    "Review Content": "review_content",
    "Verified Purchase": "verified_purchase",
    "Helpful Review Count": "helpful_review_count",
    "Manufacturer Response": "manufacturer_response",
})
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10002 entries, 0 to 10001
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   product_id             10002 non-null  object 
 1   product_name           10002 non-null  object 
 2   category               9172 non-null   object 
 3   upc_ean_code           34 non-null     object 
 4   selling_price          9895 non-null   object 
 5   model_number           8232 non-null   object 
 6   about_product          9729 non-null   object 
 7   product_specification  8370 non-null   object 
 8   technical_details      9212 non-null   object 
 9   shipping_weight        8864 non-null   object 
 10  product_dimensions     479 non-null    object 
 11  image                  10002 non-null  object 
 12  variants               2478 non-null   object 
 13  product_url            10002 non-null  object 
 14  is_amazon_seller       10002 non-null  object 
 15  cr

## Missing Data Handling

In [8]:
df.isna().sum().sort_values(ascending= False)

crawl_timestamp          10002
billing_id               10002
helpful_review_count     10002
verified_purchase        10002
review_content           10002
review_url               10002
asin                     10002
product_description      10002
sub_category             10002
brand                    10002
user_id                  10002
review_date              10002
review_rating            10002
review_title             10002
rating                   10002
manufacturer_response    10002
upc_ean_code              9968
product_dimensions        9523
variants                  7524
model_number              1770
product_specification     1632
shipping_weight           1138
category                   830
technical_details          790
about_product              273
selling_price              107
product_name                 0
is_amazon_seller             0
product_url                  0
image                        0
product_id                   0
dtype: int64

In [9]:
# Drop columns with > 97% missing
significant_missing = df.columns[df.isna().mean() > 0.97]
df = df.drop(columns= significant_missing)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10002 entries, 0 to 10001
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   product_id             10002 non-null  object
 1   product_name           10002 non-null  object
 2   category               9172 non-null   object
 3   selling_price          9895 non-null   object
 4   model_number           8232 non-null   object
 5   about_product          9729 non-null   object
 6   product_specification  8370 non-null   object
 7   technical_details      9212 non-null   object
 8   shipping_weight        8864 non-null   object
 9   product_dimensions     479 non-null    object
 10  image                  10002 non-null  object
 11  variants               2478 non-null   object
 12  product_url            10002 non-null  object
 13  is_amazon_seller       10002 non-null  object
dtypes: object(14)
memory usage: 1.1+ MB


In [10]:
df['selling_price'] = df['selling_price'].apply(extract_prices)
df['main_category'] = df['category'].apply(extract_main_category)
df["shipping_weight"] = df["shipping_weight"].apply(extract_weights)
df["is_amazon_seller"] = df["is_amazon_seller"].map({"Y": True, "N": False})
df[['dim_length', 'dim_width', 'dim_height', 'dim_unit']] = df['product_dimensions'].apply(extract_dimensions)
df['brand_name'] = df['product_name'].apply(extract_brand)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10002 entries, 0 to 10001
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   product_id             10002 non-null  object 
 1   product_name           10002 non-null  object 
 2   category               9172 non-null   object 
 3   selling_price          9849 non-null   float64
 4   model_number           8232 non-null   object 
 5   about_product          9729 non-null   object 
 6   product_specification  8370 non-null   object 
 7   technical_details      9212 non-null   object 
 8   shipping_weight        8863 non-null   float64
 9   product_dimensions     479 non-null    object 
 10  image                  10002 non-null  object 
 11  variants               2478 non-null   object 
 12  product_url            10002 non-null  object 
 13  is_amazon_seller       10002 non-null  bool   
 14  main_category          9172 non-null   object 
 15  di

In [12]:
# Impute selling price by main category median
price_fills = df.groupby('main_category')['selling_price'].transform('median')
glob_median = df['selling_price'].median()
df['selling_price'] = (
    df['selling_price']
    .fillna(price_fills)
    .fillna(glob_median)
)

# Impute shipping weight by main category median
weight_fills = df.groupby('main_category')['shipping_weight'].transform('median')
glob_weight = df['shipping_weight'].median()
df['shipping_weight'] = (
    df['shipping_weight']
    .fillna(weight_fills)
    .fillna(glob_weight)
)

# Compute normalized weight
df['normalized_weight'] = df.apply(
    lambda row: extract_price_per(row['selling_price'], row['shipping_weight']),
    axis= 1
)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10002 entries, 0 to 10001
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   product_id             10002 non-null  object 
 1   product_name           10002 non-null  object 
 2   category               9172 non-null   object 
 3   selling_price          10002 non-null  float64
 4   model_number           8232 non-null   object 
 5   about_product          9729 non-null   object 
 6   product_specification  8370 non-null   object 
 7   technical_details      9212 non-null   object 
 8   shipping_weight        10002 non-null  float64
 9   product_dimensions     479 non-null    object 
 10  image                  10002 non-null  object 
 11  variants               2478 non-null   object 
 12  product_url            10002 non-null  object 
 13  is_amazon_seller       10002 non-null  bool   
 14  main_category          9172 non-null   object 
 15  di

In [13]:
df.head(3).T

,0,1,2
product_id,4c69b61db1fc16e7013b43fc926e502d,66d49bbed043f5be260fa9f7fbff5957,2c55cae269aebf53838484b0d7dd931a
product_name,"DB Longboards CoreFlex Crossbow 41"" Bamboo Fib...","Electronic Snap Circuits Mini Kits Classpack, ...",3Doodler Create Flexy 3D Printing Filament Ref...
category,Sports & Outdoors | Outdoor Recreation | Skate...,Toys & Games | Learning & Education | Science ...,Toys & Games | Arts & Crafts | Craft Kits
selling_price,237.68,99.95,34.99
model_number,NaN,55324,NaN
about_product,Make sure this fits by entering your model num...,Make sure this fits by entering your model num...,Make sure this fits by entering your model num...
product_specification,Shipping Weight: 10.7 pounds (View shipping ra...,Product Dimensions: 14.7 x 11.1 x 10.2...,ProductDimensions:10.3x3.4x0.8inches|ItemWeigh...
technical_details,NaN,The snap circuits mini kits classpack provides...,show up to 2 reviews by default No longer are ...
shipping_weight,10.7,4.0,0.8
product_dimensions,NaN,14.7 x 11.1 x 10.2 inches 4.06 pounds,NaN


In [14]:
df['metadata'] = df.apply(extract_metadata, axis= 1)
df['embedding_text'] = df.apply(extract_embedding, axis= 1)

In [15]:
slice_stats = (
    df.groupby("main_category")
    .agg(
        n_products = ("product_id", "count"),
        avg_about_len = ("about_product", lambda x: x.fillna("").str.len().mean()),
        missing_dims = ("dim_length", lambda x: x.isna().mean()),
        text_coverage = ("embedding_text", lambda x: x.fillna("").str.len().mean()),
    )
    .sort_values("n_products", ascending=False)
)

filtered = slice_stats[slice_stats["n_products"] >= 200]

ranked = filtered.sort_values(
    by=["missing_dims", "text_coverage", "avg_about_len"],
    ascending=[True, False, False]
)

ranked

,n_products,avg_about_len,missing_dims,text_coverage
main_category,,,,
sports & outdoors,540,564.388889,0.627778,1538.838889
home & kitchen,708,442.727401,0.967514,1157.781073
toys & games,6662,419.123687,0.974032,1154.618283
baby products,214,417.228972,0.990654,1265.457944
"clothing, shoes & jewelry",630,324.347619,0.992063,1084.609524


In [16]:
sports = df[df["main_category"] == "sports & outdoors"]
homes = df[df["main_category"] == "home & kitchen"]

In [17]:
sports.to_csv('../data/sports_preprocessed_data.csv')
homes.to_csv('../data/homes_preprocessed_data.csv')